## Covariance Sensor Anomaly Detection

Covariance Detection is found from finding the **coviarance** of labeled data, finding the covariance of the label. We test a dataset of the sensors that have the correct labels, and another dataset which may or may not have proper sensors. If the new dataset we are comparing is properly labeled, then they should have relatively high (at least higher) correlation with the proper labelled dataset.

For the Ampere Dataset, we load the data by first getting the data from a csv file, then pulling just the sensor data using keys. The data is delimited with commas and the labels are on the first row.

In [ ]:
data = np.genfromtxt(cwd + csv_path, dtype=None, delimiter=',', names=True)
print(data)

rows = len(data)
cols = len(data[0])
print("Rows and columns of data:\nrows:", rows, "cols:", cols, '\n')

First we pull the sensor data Sorted by the P (rows of the sensors). Note that this above numpy function allows us to access labels with a key. To keep things consistent, we will have the labels on the columns of our matrix. We will transpose our matrix if need be for methods in the future, such a ```torch.cov``` method.

Let the columns of the matrix be the labels of our data and let the time unit be our rows. The matrix of our data are given by:

$$data = \begin{bmatrix}
x_{1,1} & \cdots & x_{n,1} \\
\cdots & \cdots & \cdots \\
x_{1,t} & \cdots & x_{n,t} \\
\end{bmatrix}$$

In [ ]:
# the r,t,z values of the sensors
l1 = ['r', 't', 'z']

# list of strings for the sensor labels (not necesarily in our data)
sensors_row = np.array(["MeasurementsP" + f'{x:02d}' + "C" + f'{y:02d}' + z \
    for x in range(dim[0][0], dim[0][1] + 1) \
        for y in range(dim[1][0], dim[1][1] + 1) for z in l1])
# reorder the sensors to be grouped by columns now
sensors_col = np.array(["MeasurementsP" + f'{x:02d}' + "C" + f'{y:02d}' + z \
    for y in range(dim[1][0], dim[1][1] + 1) \
        for x in range(dim[0][0], dim[0][1] + 1) for z in l1])

num_sensors = np.size(sensors_row)
print("The number of total possible sensors:\n", num_sensors)

Note that the above is an array of **strings**. We are creating an array of strings to access the data via key-value pairs. The **dimensions** is by $P \times C \times L$
$$Dimensions = P \times C \times L$$
Because we have 16 sensors for $P$ and 16 sensors for $C$ and each sensor has an $r,t,z$ coordinate, we should have a total of **$16 \times 16 \times 3 = 768$ total sensors**

Once we have our labels in an array, we need to **reshape** the array so that every $r,t,z$ sensor is in it's own subarray

In [ ]:
sensors_rtz = sensors_row.reshape(int(num_sensors/3), 3)
# order the sensors by the columns
sensors_rtz2 = sensors_col.reshape(int(num_sensors/3), 3)
sensors_row_rtz = sensors_row.reshape(int(num_sensors/(3*16)), 16, 3)
sensors_col_rtz = sensors_col.reshape(int(num_sensors/(3*16)), 16, 3)

Now we create our array, keeping the labels as the columns. We create a matrix that contains all the sensors sorted by $P$ (rows), all the sensors sorted by $C$ (columns), and then a list containing the $P$ (where ```list[0]``` contains all the P01 sensors) and a list containing the $C$.

It is important to note that we are also taking the **magnitude** of the $r,t,z$ sensors. We do this by calling ```np.linalg.norm```.

The arrays are transposed to keep the labels on the columns for consistency.

In [ ]:
# get all the sensors in the sensor array that exists in the data
sensors_array = np.transpose(np.array([data[x] for x in sensors_row if x in data.dtype.fields]).astype(float))
rows = np.size(sensors_array, axis=0)
cols = np.size(sensors_array, axis=1)
print("Rows and columns of sensors array:\nrows:", rows, "cols:", cols, '\n')

# make sure for each magnitude that the r,t,z values exists for each of them
# get only the magnitude if there exists an rtz value for that specific sensor
# transpose the matrix so that the columns are the sensors and the rows are the time
sensors_rtz_array_row = np.transpose(np.array([np.linalg.norm([data[rtz[0]], data[rtz[1]], data[rtz[2]]], axis=0) for rtz in sensors_rtz \
if rtz[0] in data.dtype.fields and rtz[1] in data.dtype.fields and rtz[2] in data.dtype.fields]).astype(float))

sensors_rtz_array_col = np.transpose(np.array([np.linalg.norm([data[rtz[0]], data[rtz[1]], data[rtz[2]]], axis=0) for rtz in sensors_rtz2 \
if rtz[0] in data.dtype.fields and rtz[1] in data.dtype.fields and rtz[2] in data.dtype.fields]).astype(float))

# each row has their own respective matrix
row_rtz_list = [np.transpose([np.linalg.norm([data[rtz[0]], data[rtz[1]], data[rtz[2]]], axis=0) for rtz in P \
if rtz[0] in data.dtype.fields and rtz[1] in data.dtype.fields and rtz[2] in data.dtype.fields]) for P in sensors_row_rtz]

# each column has their own respective matrix
col_rtz_list = [np.transpose([np.linalg.norm([data[rtz[0]], data[rtz[1]], data[rtz[2]]], axis=0) for rtz in P \
if rtz[0] in data.dtype.fields and rtz[1] in data.dtype.fields and rtz[2] in data.dtype.fields]) for P in sensors_col_rtz]
# reshape each of these columns so that the rtz gets their own respective rtz

This new array of magnitudes should be about a third of the total labels, or about $256$. However, you may see there are less sensors. This is because some sensors may be missing, and to input the magnitude in the lists, it requires all three $r,t,z$ to exists in the data.

We can now find the correlation coefficient of these matrices by comparing the sensors and the average of the values based on **time**. This is known as the **Pearson product-moment correlation coefficient matrix**

$$cov_{w}(x,y)=\frac{\sum^{N}_{i=1}(x_{i}-\bar{x})(y_{i}-\bar{y})}{N-1}$$

The correlation coefficient matrix $R$ is computed using the covariance matrix $C$ as given by $$R_{ij}=\frac{C_{ij}}{\sqrt{C_{ii}\times C_{jj}}}$$

In [ ]:
C = np.corrcoef(data_array, rowvar=False)
C[C == 0] = np.nan
print(C)

Notice that the rowvar is set to `False` due to the labels being on the columns. We can transpose this by using `matrix.T` or `np.transpose(matrix)`.

Once the matrix is created, we can plot the heat map of the values of the correlation coefficient from $-1$ to $1$. Getting the size of the matrix, we can plot a matrix of all the sensors, or subplots of the $P$ and $C$ sensors.

In [ ]:
cols = np.size(data_array, axis=1)

plt.figure(figsize=(12,12))
plt.imshow(data_array, vmin=-1.0,vmax=1.0)
plt.grid(True, alpha=0.15)
plt.colorbar()
plt.yticks(np.arange(0,cols))
plt.xticks(np.arange(0,cols))
plt.xlabel("Sensors")
plt.ylabel("Sensors")
plt.title(title)
plt.show()

In many situations, we want to plot the subplots of various $C$ and $P$ sensors to compare each correlation between the sensors at each specific row or column.

In [ ]:
methods = [None, 'none', 'nearest', 'bilinear', 'bicubic', 'spline16',
        'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric',
        'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos']

fig, axs = plt.subplots(nrows=Nr,ncols=Nc, figsize=(8, 8), subplot_kw={'xticks': [], 'yticks': []})

# for ax, interp_method in zip(axs.flat, methods):
for ax, x in zip(axs.flat, range(0,Nr*Nc)):
    ax.imshow(data_list[x], interpolation=method, cmap='viridis')
    ax.set_title(str(x + 1), fontsize=9, loc="center")
    plt.tight_layout()
    plt.suptitle(title, fontsize=12)
    plt.show()

However, it is more practical to compare two different datasets of sensors and compare the variance-covariance of these two datasets. We can create diagonal matrices and omit the lower half of the data.

In [ ]:
# get the diagonal matrix of the sensor covariance

# if the labels are on the columns, transpose the matrix
# so that each array is a sensor
if (rowvar == False):
    sensors1_data = sensors1_data.T
    sensors2_data = sensors2_data.T

s1 = np.size(sensors1_data, axis=0)
s2 = np.size(sensors2_data, axis=0)

if (s1 != s2):
    print("Unequal Dim!")

#  create a matrix for our covariance matrix
C = np.zeros((s1, s1), dtype=float)

M = np.size(C, axis=1)
for i in range(M):
    for j in range(i,M):
        C[i,j] = sensor_covariance(sensors1_data[i], sensors2_data[j])

C[C == 0] = np.nan

The covariance of each sensors is calculated individually, getting the covariance of each sensors which returns a $2 \times 2$ matrix. By pulling the upper-right (or lower-left for the inverse value), we can determine the covariance for each sensor we are comparing for each dataset.

In [ ]:
np.corrcoef(sensor1_data, sensor2_data)[0,1]

It is important we compare two dataset of similar settings (melt times, metals, etc). We want to compare the values in which the sensors are **most stable**. This method, in conjunction with **Dynamic Time Warping**, may prove useful. For now, we are using Euclidean distance of the two graphs.

In [ ]:
subplot_row_corr_list = [sensors_covariance(row_list1[i][10000:25000,:], row_list2[i][10000:25000,:], False) for i in range(0,16)]
subplot_col_corr_list = [sensors_covariance(col_list1[i][10000:25000,:],col_list2[i][10000:25000,:], False) for i in range(0,16)]